# **Alumno**: Álvaro Manuel Aparicio Morales

## **Módulo 6**: Seguridad Cuántica

I CERTIFICADO DE EXTENSIÓN UNIVERSITARIA EN COMPUTACIÓN CUÁNTICA

(2024-25)

# Elliptic Curve Digital Signature Algorithm (ECDSA)

# Setup

btclib is needed: let's install/update it and imported straight away some of its functions

In [ ]:
!pip install --upgrade btclib

from btclib.number_theory import mod_inv
from btclib.ec.curve import mult
from btclib.ec import bytes_from_point, point_from_octets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.6/188.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00


For this exercise we use secp256k1 as elliptic curve and SHA256 as hash function:

In [ ]:
from btclib.ec.curve import secp256k1 as ec
from hashlib import sha256 as hash
print(ec)

Curve
 p   = FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFE FFFFFC2F
 a   = 0
 b   = 7
 x_G = 79BE667E F9DCBBAC 55A06295 CE870B07 029BFCDB 2DCE28D9 59F2815B 16F81798
 y_G = 483ADA77 26A3C465 5DA4FBFC 0E1108A8 FD17B448 A6855419 9C47D08F FB10D4B8
 n   = FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFE BAAEDCE6 AF48A03B BFD25E8C D0364141
 cofactor = 1


Normally, hf is chosen such that its output size is roughly equal to the size of ec.n, since the overall security of the signature scheme will depend on the smallest of the two; however, the ECDSA standard support all combinations of sizes

In [ ]:
print(hash().digest_size)
print(ec.n_size)

32
32


# 1. Key Generation

**Private Key** (generated elsewhere, a fixed value here):

In [ ]:
q = 0x18E14A7B6A307F426A94F8114701E7D8E774E7F9A47E2C2035DB29A206324725
assert 0 < q < ec.n, "Invalid private key"
print("q:", q)
print("Hex(q):", hex(q))

q: 11253563012059685825953619222107823554536665569766687653799784088259834038053
Hex(q): 0x18e14a7b6a307f426a94f8114701e7d8e774e7f9a47e2c2035db29a206324725


and the corresponding **Public Key**:

In [ ]:
Q = mult(q, ec.G)
Q_hex = bytes_from_point(Q).hex()
print("Pubkey point: ",Q)
print("PubKey: ",Q_hex)


Pubkey point:  (20499248199127347513744745245647133074212871529272136658537990292998099609108, 55596051396810578486326870485040051763133240522441939960331386877950977471380)
PubKey:  022d5229cf0bf551ec7050a966e76e7326318a2c80d5c29b7d0f5d2cf4c74ca614


## Message
The message to be signed msg is first processed by hf, yielding to the so-called 'non-interactive challenge' c:

In [ ]:
msg = "Ejercicio de comprobación de firma"

# challenge is an integer modulo ec.n:
msg_hash = hash(msg.encode()).digest()
c = int.from_bytes(msg_hash, 'big') % ec.n
assert c != 0
print("c:", hex(c))

c: 0x522a568c75f86a4b828503d3ed43461a794a7c2e841d740297f2ea80d6f695f2


## Deterministic Ephemeral Key
An ephemeral key k is required for signing; it must be kept secret and never reused. A good choice is to use a deterministic key:

`k = hash(q||c)`

different for each msg, private because of q

In [ ]:
k_bytes = hash(q.to_bytes(32, 'big') + c.to_bytes(32, 'big')).digest()
k = int.from_bytes(k_bytes, 'big') % ec.n
assert 0 < k < ec.n, "Invalid ephemeral key"
print("k:", hex(k))

k: 0xdbedf61507e740abb9273dc870fee8b7bec9e626e8755235e25c12212141cde


## Signature Algorithm

Assuming $c$ is the hashed message and $q$ the private key, choose a random k and compute $ (x_1, y_1) = k \cdot G $

Then $r = x_1 \pmod{n}$ and $s = k^{-1} \cdot (c + r \cdot q) \pmod{n}$ is signature


In [ ]:
K = mult(k, ec.G)

r = K[0] % ec.n
# if r == 0 (extremely unlikely for large ec.n) go back to a different k
assert r != 0

s = mod_inv(k, ec.n) * (c + r*q) % ec.n
# if s == 0 (extremely unlikely for large ec.n) go back to a different k
assert s != 0

print("r:", hex(r))
print("s:", hex(s))

r: 0xe6917770edcfe6ffc765deb86a416969ca39e97cbc092e48dab333b54090948a
s: 0xb4ba35eb5169aa152ff9ee8eb9a2b93e4d327ab6f3422c24f66b31d843c362ac


# 3. Verify Signature

Compute

$u = c \cdot s^{-1} \pmod{n}, \quad v = r \cdot s^{-1} \pmod{n}$

Then ,

$(x_1, y_1) = u \cdot G + v \cdot Q$

Finally check if $r \equiv x_1 \pmod{n}$

Where:
* $c$: The hashed message (truncated if longer than $n$'s bit-length).
* $r$: The first part of the ECDSA signature.
* $s$: The second part of the ECDSA signature.
* $G$: The generator point on the elliptic curve.
* $Q$: The signer's public key.
* $n$: The order of the group generated by $G$.
* $(x_1, y_1)$: The resulting point on the elliptic curve.

In [ ]:
Q_hex=input("Paste the public key")
Q=point_from_octets(bytes.fromhex(Q_hex))
msg = input("Paste the message")
msg_hash = hash(msg.encode()).digest()
r = int(input("Paste r"),0)
s = int(input("Paste s"),0)

c = int.from_bytes(msg_hash, 'big') % ec.n
s_inv = mod_inv(s, ec.n)
u = c*s_inv % ec.n
v = r*s_inv % ec.n
assert u != 0
assert v != 0
U = mult(u, ec.G)
V = mult(v, Q)
x, y = ec.add(U, V)
print(r == x % ec.n)

Paste the public key022d5229cf0bf551ec7050a966e76e7326318a2c80d5c29b7d0f5d2cf4c74ca614
Paste the messageEjercicio de comprobación de firma
Paste r0xe6917770edcfe6ffc765deb86a416969ca39e97cbc092e48dab333b54090948a
Paste s0xb4ba35eb5169aa152ff9ee8eb9a2b93e4d327ab6f3422c24f66b31d843c362ac
True


# Key recovery by nonce reuse

## Second Message
A second different message to be signed and its corresponding challenge:

In [ ]:
msg2 = "and Paolo is right to be afraid"

# challenge is an integer modulo ec.n:
msghd2 = hash(msg2.encode()).digest()
c2 = int.from_bytes(msghd2, 'big') % ec.n
assert c2 != 0
print("c2:", hex(c2))

c2: 0x7adb91982ec03ef87efcae7f0199aefa231d8855e0bd03319460e58c0bd18049


## The Mistake
Reuse the same ephemeral key as the previous message:

In [ ]:
#very bad! Never reuse an ephemeral key!!!
k2 = k
print("k :", hex(k))
print("k2:", hex(k2))

k : 0xdbedf61507e740abb9273dc870fee8b7bec9e626e8755235e25c12212141cde
k2: 0xdbedf61507e740abb9273dc870fee8b7bec9e626e8755235e25c12212141cde


## Sign Second Message

In [ ]:
K2 = mult(k2, ec.G)

r = K2[0] % ec.n
# if r == 0 (extremely unlikely for large ec.n) go back to a different k
assert r != 0

s2 = mod_inv(k2, ec.n) * (c2 + r*q) % ec.n
# if s2 == 0 (extremely unlikely for large ec.n) go back to a different k
assert s2 != 0

print("r :", hex(r))
print("s2:", hex(s2))

r : 0xe6917770edcfe6ffc765deb86a416969ca39e97cbc092e48dab333b54090948a
s2: 0x52417c3e63faddf8209ecbb51d58c6b5fb6f138a1fdd39d681c2833329ad8bc0


## Verify Second Signature

In [ ]:
w = mod_inv(s2, ec.n)
u = c2*w % ec.n
v = r*w % ec.n
assert u != 0
assert v != 0
U = mult(u, ec.G)
V = mult(v, Q)
x, y = ec.add(U, V)
print(r == x % ec.n)

True


# Exercise
Because of the ephemeral key reuse is possible to calculate the private key from the 2 signatures.

In [ ]:
# forget k, k2, q
k=k2=q=0
q_rev=0
# solve the problem of calculating q:
# k
# q

print("Introduce the following values:")
r = int(input("Paste r value: "), 0)
s1 = int(input("Paste s1: "), 0)
s2 = int(input("Paste s2: "), 0)
c1 = int(input("Paste c1: "), 0)
c2 = int(input("Paste c2: "), 0)


signature_diff = (s1 - s2) % ec.n
c_diff =  c1 - c2 % ec.n
s_diff_inv = mod_inv(signature_diff, ec.n)
k = (c_diff * s_diff_inv) % ec.n


q_rev = ((s1 * k - c1) * mod_inv(r, ec.n)) % ec.n

# Imprimir la clave privada recuperada
print("La clave privada recuperada es:")
print(hex(q_rev))
print(mult(q_rev, ec.G) == Q) # check it is the correct private key

Introduce the following values:
Paste r value: 0xe6917770edcfe6ffc765deb86a416969ca39e97cbc092e48dab333b54090948a
Paste s1: 0xb4ba35eb5169aa152ff9ee8eb9a2b93e4d327ab6f3422c24f66b31d843c362ac
Paste s2: 0x52417c3e63faddf8209ecbb51d58c6b5fb6f138a1fdd39d681c2833329ad8bc0
Paste c1: 0x522a568c75f86a4b828503d3ed43461a794a7c2e841d740297f2ea80d6f695f2
Paste c2: 0x7adb91982ec03ef87efcae7f0199aefa231d8855e0bd03319460e58c0bd18049
La clave privada recuperada es:
0x18e14a7b6a307f426a94f8114701e7d8e774e7f9a47e2c2035db29a206324725
True


# Ejemplo de prueba

r: 0xe6917770edcfe6ffc765deb86a416969ca39e97cbc092e48dab333b54090948a


s1: 0xb4ba35eb5169aa152ff9ee8eb9a2b93e4d327ab6f3422c24f66b31d843c362ac

s2: 0x52417c3e63faddf8209ecbb51d58c6b5fb6f138a1fdd39d681c2833329ad8bc0

c1: 0x522a568c75f86a4b828503d3ed43461a794a7c2e841d740297f2ea80d6f695f2

c2: 0x7adb91982ec03ef87efcae7f0199aefa231d8855e0bd03319460e58c0bd18049